In [1]:
# Uninstall fastai for solving dependence problems
!pip uninstall fastai -y
# Install packages without internet
!pip install ../input/packages/torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/packages/torchvision-0.8.2-cp37-cp37m-manylinux1_x86_64.whl

Found existing installation: fastai 2.0.19
Uninstalling fastai-2.0.19:
  Successfully uninstalled fastai-2.0.19
Processing /kaggle/input/packages/torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
Processing /kaggle/input/packages/torchvision-0.8.2-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.7.0
    Uninstalling torchvision-0.7.0:
      Successfully uninstalled torchvision-0.7.0


In [2]:
import sys
sys.path.append('../input/repvggmodels/')
sys.path.append('../input/vision-transformer-pytorch/VisionTransformer-Pytorch')

from repvgg import RepVGG, create_RepVGG_B3, create_RepVGG_B3g4, repvgg_model_convert
from vision_transformer_pytorch import VisionTransformer

In [3]:
import os
import cv2
import time
import random
import joblib
import sklearn
import warnings
import multiprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from skimage import io
from sklearn import metrics
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
from IPython.display import display
from catalyst.data.sampler import BalanceClassSampler

import torch
import torch.nn.functional as F
import torchvision
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.nn.modules.loss import _WeightedLoss
from torchvision import models
from torchvision import transforms

from albumentations.pytorch import ToTensor, ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,
    ShiftScaleRotate, CenterCrop, Resize)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [5]:
CFG = {
    'seed'       : 42,
    'fold'       : 0 if len(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))==1 else 99,
    'tta'        : 1 if len(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))==1 else 4,
    'img_size'   : 400,
    'valid_bs'   : 32,
    'num_workers': multiprocessing.cpu_count(),
    'device'     : 'cuda:0'}

In [6]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

# Inference

In [7]:
class CassavaDataset(Dataset):
    def __init__(self, df, data_root, 
                 transforms=None, 
                 output_label=True, 
                 one_hot_label=False):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms  = transforms
        self.data_root   = data_root
        self.output_label  = output_label
        self.one_hot_label = one_hot_label
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        img = get_img(f"{self.data_root}/{self.df.loc[index]['image_id']}")
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img

In [8]:
def get_inference_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [9]:
class FFN(nn.Module):
    def __init__(self, backbone=None):
        super(FFN, self).__init__()
        self.backbone = backbone
        self.lr1      = nn.Linear(1000, 256)
        self.relu     = nn.ReLU()
        self.dropout  = nn.Dropout(0.5)
        self.lr2      = nn.Linear(256, 5)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.relu(self.lr1(x))
        x = self.dropout(x)
        x = self.lr2(x)
        return x
    
class ViT(nn.Module):
    def __init__(self, backbone=None):
        super(ViT, self).__init__()
        self.backbone = backbone
        
    def forward(self, x):
        x = self.backbone(x)
        return x

In [10]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [11]:
def predict(df, data_path, model_name, model_path, backbone, load=True):
    results = np.zeros((len(os.listdir(data_path)), 5))
    models  = [m for m in os.listdir(model_path) if m.find("csv")==-1]
    for model_file in models:
        print(model_file)
        device  = torch.device(CFG['device'])
        dataset = CassavaDataset(df, data_path, transforms=get_inference_transforms(), output_label=False)

        data_loader = torch.utils.data.DataLoader(
            dataset, 
            batch_size =CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False)

        model = model_name(backbone)
        if load:
            if CFG['device'] == "cpu":
                model.load_state_dict(torch.load(f"{model_path}{model_file}", map_location="cpu"))
            else:
                model.load_state_dict(torch.load(f"{model_path}{model_file}"))
        backbone.to(device)
        model.to(device)

        preds = []
        with torch.no_grad():
            for _ in range(CFG['tta']):
                preds += [(1/CFG['tta'])*inference_one_epoch(model, data_loader, device)]
        preds    = np.sum(preds, 0)
        results += preds
        del model
        torch.cuda.empty_cache()
        
        if CFG['fold'] == 0:
            return results
        
    return results / len(models)

In [12]:
test_path  = '../input/cassava-leaf-disease-classification/test_images/'
test = pd.DataFrame()
test['image_id'] = list(os.listdir(test_path))
model_path = '../input/cassava-models-trained-with-noise-labels/'
backbone   = create_RepVGG_B3g4(deploy=True)

results_from_noise_model = predict(test, test_path, FFN, model_path, backbone)

cnn_with_noise_label_fold_0


100%|██████████| 1/1 [00:00<00:00,  1.31it/s]


In [13]:
model_path = '../input/cassava-noised-label-data/'
backbone   = create_RepVGG_B3g4(deploy=True)

results_from_clean_model = predict(test, test_path, FFN, model_path, backbone)

tuned_cnn_fold_0


100%|██████████| 1/1 [00:00<00:00,  7.64it/s]


In [14]:
model_path = '../input/vit-model-1/'
backbone   = VisionTransformer.from_name('ViT-B_16', num_classes=5)
backbone.load_state_dict(torch.load(model_path+"/ViT-B_16.pt"))

CFG["img_size"] = 384

results_from_vit = predict(test, test_path, ViT, model_path, backbone, load=False)

ViT-B_16.pt


100%|██████████| 1/1 [00:00<00:00,  9.22it/s]


In [15]:
results = 0.4*results_from_noise_model + 0.2*results_from_clean_model + 0.4*results_from_vit

In [16]:
test['label'] = np.argmax(results,1)
test.head()

,image_id,label
0,2216849948.jpg,4


In [17]:
test.to_csv('submission.csv', index=False)